# Inference in Hidden Markov Models (HMMs)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
%matplotlib inline

Suppose we have a first order Hidden Markov Model with hidden states $h_t$ and observed states $v_t$. We may be interested in the probability of the latent variables $h_t$ given the observations $v_t$, inference problems of this type can be classified as follows:

* **Filtering:** Inferring the probability of the present hidden state at time $t$ given observations to that time - $p(h_t \ | \ v_{1:t})$
* **Smoothing:** Inferring the probability of a past hidden state at some time $t$ given observations up to and in the future of t - $p(h_t \ | \ v_{1:s})$ for $t < s$
* **Most likely path:** Inferring the most likely sequence of hidden states given a set of observations - $argmax_{\ h1:t} \ p(h_{1:t} \ | \ v_{1:t})$

<img src="http://recognize-speech.com/images/barber2014/barber2014.fig.23.4.png", width="50%">

Let's make a 100 timestep HMM such that each hidden state $h_t$ is binary (i.e. 0 or 1), while the observed states $v_t$ may take 3 values (0, 1 or 2). Next we need to specify the marginal distribution $P(h_1)$ and the transition matrix $P(h_t = i \ | \ h_{t-1} = j) = P_{ij}$. We also need an emission matrix to generate observations relating to our hidden states, we define this matrix as $A_{ij} = P(v_t = i \ | \ h_t = j)$.

In [2]:
N = 2 # number of hidden states
M = 3 # number of observed states
T = 100 # number of timesteps

In [3]:
# Create random marginal for h1
pi = np.random.rand(N)
pi = pi/np.sum(pi)

# Create transition matrix 
P = np.array([[0.7,0.2],[0.3,0.8]])

# Create emission matrix
A = np.array([[1./3,1./2],[1./3,0.0],[1./3,1./2]])

In [4]:
print("Marginal of h1 = \n", pi)
print("Transition matrix P = \n",P)
print("Emission Matrix A = \n", A)

Marginal of h1 = 
 [ 0.3287607  0.6712393]
Transition matrix P = 
 [[ 0.7  0.2]
 [ 0.3  0.8]]
Emission Matrix A = 
 [[ 0.33333333  0.5       ]
 [ 0.33333333  0.        ]
 [ 0.33333333  0.5       ]]


Just to ensure we're all on the same page, let's describe what's happening with these. The marginal matrix defines our probability of being in each possible hidden state at step 1, the emission matrix then uses the probabilities relating to this hidden state to generate observations, finally at the next timestep we transition using the matrix p based on our previous hidden state.

In [5]:
def generate_data(pi, P, A, T):
    # Arrays for hidden and observed states
    H = np.zeros(T, dtype = int)
    V = np.zeros(T, dtype = int)
    n = len(pi)
    m = A.shape[0]
    for t in range(T):
        # Generate transition for next timestep
        if t == 0:
            H[t] = np.random.choice(n, p = pi)
        else:
            H[t] = np.random.choice(n, p = P[:,H[t-1]])
        # Generate observed state based on current hidden state
        V[t] = np.random.choice(m, p = A[:,H[t]])
    return (H,V)

In [6]:
# Generate data for 100 timesteps
(H,V) = generate_data(pi, P, A, T)

In [7]:
print("Hidden states: \n", H)
print("Observed states: \n", V)

Hidden states: 
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 0 0 0 1 1 1 1 1 1 1 1]
Observed states: 
 [0 2 0 0 2 0 0 2 2 0 0 0 2 2 0 2 1 1 0 2 0 0 0 1 1 2 1 0 2 0 0 2 1 0 2 0 2
 2 2 0 2 2 2 0 2 0 2 2 0 0 2 2 2 2 0 0 2 2 2 0 1 0 2 1 1 0 0 2 0 2 0 1 0 2
 0 2 0 0 0 2 0 0 2 2 1 1 2 2 2 1 0 0 2 0 2 2 2 2 0 2]


Now suppose we are interesting in filtering, i.e. $p(h_t | v_{1:t}) \propto p(h_t, v_{1:t})$. Computing this directly would require marginalizing over all possible previous hidden states $h_{1:t-1}$, the number of which grows exponentially with t. Instead we can make use of the forward algorithm, which takes advantage of the conditional independence rules of the HMM to perform marginalisation recursively. To demonstrate the recursion, let

$$\alpha_t(h_t) = p(h_t, v_{1:t}) = \sum_{h_{t-1}} p(h_t, h_{t-1}, v_{1:t})$$

Using the chain rule to expand $p(h_t, h_{t-1}, v_{1:t})$, we can write

$$\alpha_t(h_t) = \sum_{h_{t-1}} p(v_t \ | \ h_t, h_{t-1}, v_{1:t-1} ) \ p(h_t \ | \ h_{t-1}, v_{1:t-1}) \ p(h_{t-1}, v_{1:t-1})$$

Because $v_t$ is conditionally independent of everything but $h_t$, and $h_t$ is conditionally independent of everything but $h_{{t-1}}$, this simplifies to

$$\alpha_t(h_t) = p(v_t \ | \ h_t) \sum_{h_{t-1}} p(h_t \ | \ h_{t-1}) \ \alpha_{t-1}(h_{t-1})$$

where $p(v_t \ | \ h_t)$ is the emission matrix and $p(h_t \ | \ h_{t-1})$ is the transition matrix. Here the total number of summations is $(t-1)k$ for k hidden states, rather than $k^t$.

In [8]:
def forward_messages(pi, P, A, V):
    T = len(V)
    alpha = np.zeros([N, T])
    alpha[:, 0] = A[V[0], :] * pi
    for t in range(1,T):
        alpha[:, t] = A[V[t]] * np.sum(P * alpha[:,t-1], axis=1)
    return alpha

In [10]:
# Calculate forward messages and grab last alpha
alphas = forward_messages(pi, P, A, V)
last_alpha = alphas[:, -1]

print("P(h100 | v1:100) = \n", last_alpha/np.sum(last_alpha))
print("True h100 is", H[-1])

P(h100 | v1:100) = 
 [ 0.23811172  0.76188828]
True h100 is 1


Suppose now that we are interested in inferring the hidden state at some arbitrary time in the past, i.e. $p(h_t \ | \ v_{1:T})$ for $t<T$. We therefore want to evaluate

$$p(h_t \ | \ v_{1:T}) = p(h_t \ | \ v_{1:t}) \ p(v_{t+1:T} \ | \ h_t)$$

Where the left hand terms $p(h_t \ | \ v_{1:t})$ are the forward recursions we calculated previously. We therefore need $p(v_{t+1:T} \ | \ h_t)$, which using a similar approach to above we calculate the backwards recursions as:

$p(v_{t+1:T} \ | \ h_t) = \sum_{h_{t+1}} p(v_{t+1:T}, h_{t+1}\ | \ h_t) = \sum_{h_{t+1}} p(v_{t+2:T} \ | \ h_{t+1}) \ p(v_{t+1} \ | \ h_{t+1}) \ p(h_{t+1} \ | \ h_t)$

Which has the following recursion

$\beta_t(h_t) = \sum_{h_{t+1}} \beta_{t+1}(h_{t+1}) \ p(v_{t+1} \ | \ h_{t+1}) \ p(h_{t+1} \ | \ h_t)$

where $\beta_T(h_T) = 1$. 

The marginal $p(h_t \ | \ v_{1:T})$ is then calculated by taking the product of the forward and backward messages to $h_t$ and normalising.

In [11]:
def backward_messages(pi, P, A, V):
    T = len(V)
    beta = np.zeros([N, T])
    beta[:,T-1] = 1.0
    for t in range(T-1, 0, -1):
        # Take care of summation using matrix multiplication
        beta[:, t-1] = beta[:,t] * A[V[t]] @ P
    return beta

In [15]:
# calculate backwards messages
betas = backward_messages(pi,P,A,V)

# calculate forward and backward messages to h1
print("P(h1   | v1) = \n", alphas[:,0]/np.sum(alphas[:,0]))
print("P(v2:T | h1) = \n", betas[:,0]/np.sum(betas[:,0]))

# take the product of these messages and normalise
Pt = alphas[:,0] * betas[:,0]
print("P(h1   | v1:T) = \n", Pt/np.sum(Pt))

P(h1   | v1) = 
 [ 0.24614844  0.75385156]
P(v2:T | h1) = 
 [ 0.41265116  0.58734884]
P(h1   | v1:T) = 
 [ 0.18659677  0.81340323]


In [13]:
def marginal(alphas, betas, ht):
    'Return normalised product of forward and backward messages for some ht'
    Pt = alphas[:, ht] * betas[:, ht]
    return Pt/np.sum(Pt)

In [14]:
'Some probabilities we can now calculate'
print("P(h100 | v1:100) = \n", marginal(alphas, betas, 99))
print("P(h50  | v1:100) = \n", marginal(alphas, betas, 49))
print("P(h1   | v1:100) = \n", marginal(alphas, betas, 0))

P(h100 | v1:100) = 
 [ 0.23811172  0.76188828]
P(h50  | v1:100) = 
 [ 0.17996299  0.82003701]
P(h1   | v1:100) = 
 [ 0.18659677  0.81340323]
